# Travelling Salesperson Problem solved using genetic algorithms

In [1]:
# Imports 
import numpy as np
import random

np.random.seed(42)
# Imports 
import numpy as np
import random
import pandas as pd
from geopy import distance,geocoders,Nominatim # to calculate distance on the surface
import folium
from datetime import datetime
from datetime import datetime

In [2]:
# Parameters
n_cities = 10

n_population = 50

mutation_rate = 0.3
df = pd.read_csv('output.csv')


In [3]:
# Generating a list of coordenades representing each city
geolocator = geocoders.Nominatim(user_agent="my_email@myserver.com")
coordinates_list=[]
names_list = np.array(df['Street'].head(n_cities))
for city in names_list:
    location = geolocator.geocode(city)
    coordinates_list.append(list((location.latitude, location.longitude)))
    
cities_dict = { x:y for x,y in zip(names_list,coordinates_list)}

# Function to compute the distance between two points
def compute_city_distance_coordinates(a,b):
    return distance.distance((a[0], a[1]), (b[0],b[1])).km

def compute_city_distance_names(city_a, city_b, cities_dict):
    return compute_city_distance_coordinates(cities_dict[city_a], cities_dict[city_b])

coordinates_list

[[50.85798545, 5.6969881818221095],
 [50.776351, 6.083862],
 [50.8775239, 5.981506585454879],
 [50.9673322, 5.8277007],
 [50.9974235, 5.8666627],
 [51.1073016, 5.8719765],
 [50.735851, 7.10066],
 [50.889865, 5.8575404],
 [50.8141622, 5.6692203],
 [52.2723505, 7.2796275]]

## 1. Create the first population set
We randomly shuffle the cities N times where N=population_size

In [4]:
# First step: Create the first population set
def genesis(city_list, n_population):

    population_set = []
    for i in range(n_population):
        #Randomly generating a new solution
        sol_i = city_list[np.random.choice(list(range(n_cities)), n_cities, replace=False)]
        population_set.append(sol_i)
    return np.array(population_set)

population_set = genesis(names_list, n_population)
#population_set

## 2. Evaluate solutions fitness
The solutions are defined so that the first element on the list is the first city to visit, then the second, etc. and the last city is linked to the first.
The fitness function needs to compute the distance between subsequent cities.

In [5]:
def fitness_eval(city_list, cities_dict):
    total = 0
    for i in range(n_cities-1):
        a = city_list[i]
        b = city_list[i+1]
        total += compute_city_distance_names(a,b, cities_dict)
    return total

In [6]:
def get_all_fitnes(population_set, cities_dict):
    fitnes_list = np.zeros(len(population_set))

    #Looping over all solutions computing the fitness for each solution
    for i in  range(len(population_set)):
        fitnes_list[i] = fitness_eval(population_set[i], cities_dict)

    return fitnes_list

fitnes_list = get_all_fitnes(population_set,cities_dict)
fitnes_list

array([569.54434638, 560.5706764 , 510.39577204, 412.30583968,
       470.36060249, 599.73634939, 507.05941247, 605.68968397,
       691.39853516, 663.83986731, 534.11247006, 555.86990342,
       592.05253487, 589.87370991, 466.52173509, 556.5600469 ,
       503.31869899, 585.32983421, 608.60945219, 637.49153198,
       404.68555277, 627.41726867, 577.9223998 , 589.94675899,
       407.55300235, 582.01466331, 672.58403221, 502.16731786,
       573.02183409, 637.02779277, 592.27723143, 481.34620784,
       646.05837946, 552.61001851, 643.48191056, 517.43150587,
       660.58452059, 686.435529  , 628.02824572, 631.63160407,
       654.03789148, 587.07178013, 667.94509624, 561.8781111 ,
       531.90630901, 449.84123783, 651.05564001, 567.65066189,
       630.48345118, 616.78691243])

# 3. Progenitors selection
I will select a new set of progenitors using the Roulette Wheel Selection. Generates a list of progenitor pairs where N= len(population_set) but at each position there are two solutions to merge

In [7]:
# ORG
def progenitor_selection(population_set,fitnes_list):
    total_fit = fitnes_list.sum()
    prob_list = fitnes_list/total_fit
    
    #Notice there is the chance that a progenitor. mates with oneself
    progenitor_list_a = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    progenitor_list_b = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    
    progenitor_list_a = population_set[progenitor_list_a]
    progenitor_list_b = population_set[progenitor_list_b]
    
    
    return np.array([progenitor_list_a,progenitor_list_b])


progenitor_list = progenitor_selection(population_set,fitnes_list)
progenitor_list[0][2]

array(['sittard', 'geleen', 'kanne', 'maastricht', 'heerlen', 'echt',
       'bonn', 'hulsberg', 'ohne', 'aachen'], dtype=object)

In [8]:


def progenitor_selection_by_rank(population_set, fitnes_list, n_pairs):
    n = len(population_set)
    rank_sum = n * (n + 1) / 2
    prob_list = [i/rank_sum for i in range(1,n+1)][::-1]

    f = fitnes_list.argsort()
    #Notice there is the chance that a progenitor. mates with oneself
    progenitor_list_a = np.random.choice(f, n_pairs, p=prob_list, replace=True)
    progenitor_list_b = np.random.choice(f, n_pairs, p=prob_list, replace=True)
    
    progenitor_list_a = population_set[progenitor_list_a]
    progenitor_list_b = population_set[progenitor_list_b]
    
    
    return np.array([progenitor_list_a,progenitor_list_b])


progenitor_list = progenitor_selection_by_rank(population_set,fitnes_list, len(population_set))
progenitor_list[0][2]

array(['sittard', 'maastricht', 'aachen', 'ohne', 'bonn', 'kanne',
       'hulsberg', 'geleen', 'heerlen', 'echt'], dtype=object)

# 4. Mating
For each pair of  parents we'll generate an offspring pair. Since we cannot repeat cities what we'll do is copy a random chunk from one progenitor and fill the blanks with the other progenitor.

In [9]:
def mate_progenitors(prog_a, prog_b):
    offspring = prog_a[0:5]

    for city in prog_b:

        if not city in offspring:
            offspring = np.concatenate((offspring,[city]))

    return offspring
            
    
    
def mate_population(progenitor_list):
    new_population_set = []
    for i in range(progenitor_list.shape[1]):
        prog_a, prog_b = progenitor_list[0][i], progenitor_list[1][i]
        offspring = mate_progenitors(prog_a, prog_b)
        new_population_set.append(offspring)
        
    return new_population_set

new_population_set = mate_population(progenitor_list)
new_population_set[0]

array(['maastricht', 'aachen', 'kanne', 'echt', 'geleen', 'hulsberg',
       'bonn', 'heerlen', 'sittard', 'ohne'], dtype=object)

# 5. Mutation
Now for each element of the new population we add a random chance of swapping

In [10]:
def mutate_offspring(offspring):
    for q in range(int(n_cities*mutation_rate)):
        a = np.random.randint(0,n_cities)
        b = np.random.randint(0,n_cities)

        offspring[a], offspring[b] = offspring[b], offspring[a]

    return offspring
    
    
def mutate_population(new_population_set):
    mutated_pop = []
    for offspring in new_population_set:
        mutated_pop.append(mutate_offspring(offspring))
    return mutated_pop

mutated_pop = mutate_population(new_population_set)
mutated_pop[0]

array(['hulsberg', 'geleen', 'kanne', 'sittard', 'aachen', 'maastricht',
       'bonn', 'heerlen', 'echt', 'ohne'], dtype=object)

# 6. Stopping
To select the stopping criteria we'll need to create a loop to stop first. Then I'll set it to loop at 1000 iterations.

In [20]:
best_solution = [-1,np.inf,np.array([])]

n_elitism=1
mutation_rate = 0.3

n_gens = 1000

for i in range(n_gens):
    if i%100==0: print(i, fitnes_list.min(), fitnes_list.mean(), datetime.now().strftime("%d/%m/%y %H:%M"))
    
    
    if i == 0:
        #Compute all fitness
        fitnes_list = get_all_fitnes(mutated_pop,cities_dict)
    else:
        #Compute new fitness
        fitnes_list2 = get_all_fitnes(mutated_pop2,cities_dict)
        fitnes_list = np.concatenate((fitnes_list[fitnes_list.argsort()][:n_elitism],fitnes_list2))
    
    #Saving the best solution
    if fitnes_list.min() < best_solution[1]:
        best_solution[0] = i
        best_solution[1] = fitnes_list.min()
        best_solution[2] = np.array(mutated_pop)[fitnes_list.min() == fitnes_list]
    
    #This can be set as progressive elitism
    n_elitism= min(len(new_population_set), int(np.log(i+1)))
    mutation_rate = i/n_gens
    
    progenitor_list = progenitor_selection_by_rank(population_set,fitnes_list, len(population_set)- n_elitism)
    offspring = mate_population(progenitor_list)
    
    mutated_pop2 = mutate_population(offspring)
    
    mutated_pop = np.concatenate((np.array(mutated_pop)[fitnes_list.argsort()[:n_elitism]],mutated_pop2[:len(population_set)-n_elitism]))
    


0 322.4554206704063 556.5252759196784 15/01/23 01:34
100 357.7407957526576 553.2804369791342 15/01/23 01:34
200 322.4554206704063 550.5617747959408 15/01/23 01:34
300 322.4554206704063 557.6335558545558 15/01/23 01:34
400 322.4554206704063 540.383339197704 15/01/23 01:35
500 322.4554206704063 552.261729507881 15/01/23 01:35
600 322.4554206704063 536.3121389248963 15/01/23 01:35
700 322.4554206704063 532.5200437883846 15/01/23 01:35
800 322.4554206704063 542.0958175671378 15/01/23 01:35
900 322.4554206704063 545.9969466638361 15/01/23 01:35


In [21]:
#645
best_road = best_solution[2][0]
best_road

array(['ohne', 'echt', 'sittard', 'kanne', 'maastricht', 'geleen',
       'hulsberg', 'heerlen', 'aachen', 'bonn'], dtype=object)

In [22]:
m = folium.Map(location=cities_dict[best_road[0]], zoom_start=12, tiles="Stamen Terrain")
loc = []
for el in best_road:
  loc.append(cities_dict[el])
  folium.Marker(
      cities_dict[el], popup=f"<i>{el}</i>"
  ).add_to(m)
loc.append(cities_dict[best_road[0]])
folium.PolyLine(loc,
                color='blue',
                weight=4,
                opacity=0.9).add_to(m)
m

In [23]:
best_road = best_solution[2][0]
best_road


array(['ohne', 'echt', 'sittard', 'kanne', 'maastricht', 'geleen',
       'hulsberg', 'heerlen', 'aachen', 'bonn'], dtype=object)